In [21]:
import pandas as pd
import numpy as np

# Specify the file path
file_path = 'redshifts.txt'

# Load the DataFrame from the CSV file
df = pd.read_table(file_path, delimiter='\t')  # Assuming the file is tab-delimited, adjust the delimiter if needed

In [25]:
import re

def filter_redshift_by_grb_names(index_file_path, redshift_file_path, output_file_with_redshift, output_file_without_redshift):
    # Read the GRB names from index.txt
    with open(index_file_path, 'r') as index_file:
        grb_names_to_select = [line.strip() for line in index_file if line.strip()]

    # Read the redshift.txt file
    redshift_df = pd.read_csv(redshift_file_path, delimiter='\t')

    # Filter rows based on the selected GRB names
    filtered_redshift_df = redshift_df[redshift_df['GRB'].isin(grb_names_to_select)]

    # Extract numerical values from 'PhotonIndex' column
    filtered_redshift_df['PhotonIndex'] = filtered_redshift_df['PhotonIndex'].apply(
        lambda x: float(x.split(',')[0]) if isinstance(x, str) and ',' in x else 'NaN'
    )

    # Sort the DataFrame by the 'GRB' column
    filtered_redshift_df = filtered_redshift_df.sort_values(by='GRB')

    # Reset the indices
    filtered_redshift_df = filtered_redshift_df.reset_index(drop=True)

    # Select certain rows based on a condition
    selected_columns = ['GRB','Redshift_crosscheck', 'T90', 'Gamma', 'Fluence', 'FluenceError', 'PhotonIndex', 'PhotonIndexError', 'NH', 'PeakFlux', 'PeakFluxError']  # Adjust the condition as needed

    filtered_redshift_df = filtered_redshift_df[selected_columns]

    # Input 'NaN' string
    filtered_redshift_df = filtered_redshift_df.fillna('NaN')

    # Add 'log10Ta', 'log10Fa', 'alpha', and respective error columns
    # GRB , Ta, TaErr, Fa, FaErr, alpha, alphaErr
    fit_params = [['220611A', '220521A', '210323A', '210517A', '201014A', '200411A', '221226B', '221110A', '210726A', '210610A', '210222B', '210104A', '210222B', '210104A', '201104B', '201024A', '201020A', '200205B', '191011A', '21722A', '210731A', '211024B', '230414B', '230325A', '191031D', '190829A', '201221A', '190719C', '230506C', '230818A', '231117A', '230328', '230116D'],
                  [1.12971557, 3.30131796, 4.32838093, 3.05765100, 5.54700512, 1.96654768, 0.47461698, 2.20438137, 2.72682434, 3.15453451, 3.24642494, 3.19233419, 3.24642494, 3.19233419, 2.72159651, 3.01817341, 3.96908872, 4.53640048, 3.15321333, 3.94283864, 4.80200435, 3.43921565, 4.87925740, 3.78230810, 2.322042249, 3.40640655, 4.54931529, 4.88123, 4.25859, 3.9564, 3.78095, 3.66388, 3.79761],
                  [0.81046481, 0.13206168, 0.09651687, 0.32132806, 0.16137784, 0.25685294, 0.14697180, 0.68618385, 0.14755536, 1.28215864, 0.04927183, 0.00889405, 0.04927183, 0.00889405, 0.02972805, 0.63285167, 0.04826492, 0.08886983, 0.06597695, 0.06056870, 0.11699725, 0.01167528, 0.06954900, 0.20042492, 0.06348750, 0.00974555, 0.02804575, 0.194300, (4.25859-4.14018), (3.9564-3.82846), (3.78095-3.76714), (3.66388-3.63157), (3.79761-3.65937)], 
                  [-12.4941008, -10.6276115,  -11.9389232, -10.8415194, -12.43719871, -10.9488278, -7.35134395, -9.52540051, -11.1684430, -10.4044334, -10.4520599, -10.4044334, -10.4520599, -9.42347066, -9.38944702, -10.4667906, -11.0900559, -11.7337376, -10.8132081, -14.4834179, -11.9404653, -8.56429146, -11.9505263, -10.9066853, -9.63606913, -8.58538291, -12.0789806, -11.257, -11.4531, -11.0883, -6.90879, -10.1038, -11.1278], 
                  [0.22640221, 0.16334814, 0.31176236, 0.25872455, 0.33104417, 1.04551800, 0.11871169, 0.67651915, 0.10610593, 0.85146569, 0.04184217, 0.85146569, 0.04184217, 0.00753937, 0.02110237, 0.58074222, 0.04915322, 0.10035241, 0.005719327, 0.267760622, 0.16085961, 0.01916437, 0.11074106, 0.18754550, 0.11503072, 0.00773590, 0.04385192, 0.2175, (-11.4531+11.5554), (-11.0883+11.2927), (-6.90879+6.96363), (-10.1038+10.1305), (-11.1278+11.2292)], 
                  [2.26249671, 1.71067512, 3.40011162, 1.21043545, 2.73545391, 0.84579687, 1.32089449, 1.02202171, 0.62132497, 1.08167850, 1.37828029, 1.08167850, 1.37828029, 1.19649204, 1.19537497, 0.99913283, 1.55158826, 1.56884903, 1.30602449, 1.60481787, 1.74153741, 1.93633006, 2.06160972, 0.931411844, 1.90294890, 1.18850034, 2.93923332, 1.55953, 1.14181, 1.39195, 0.683779, 1.20938, 1.57289], 
                  [0.50304248, 0.12291833, 0.68301762, 0.08060357, 0.94410954, 0.30665794, 0.04333399, 0.03483546, 0.14755536, 0.07850685, 0.005516005, 0.07850685, 0.05516005, 0.01279355, 0.02348264, 0.03486451, 0.08306159, 0.19267266, 0.05756962, 0.06764959, 0.27863334, 0.00965772, 0.21988187, 0.05714513, 0.05586376, 0.00361642, 0.004540267, 0.21085, (1.14181-1.02882), (1.39195-1.1948), (0.683779-0.658857), (1.20938-1.18517), (1.57289-1.33798)]]

    # Loop through each set of parameters in fit_params
    for i in range(len(fit_params[0])):
        grb_name = fit_params[0][i]
        ta_value = fit_params[1][i]
        ta_err_value = fit_params[2][i]
        fa_value = fit_params[3][i]
        fa_err_value = fit_params[4][i]
        alpha_value = fit_params[5][i]
        alpha_err_value = fit_params[6][i]

        # Check if GRB name is in the filtered DataFrame
        if grb_name in filtered_redshift_df['GRB'].values:

            # Add values to the DataFrame
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'log10Ta'] = ta_value
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'log10TaErr'] = ta_err_value
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'log10Fa'] = fa_err_value
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'log10FaErr'] = fa_value
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'alpha'] = alpha_value
            filtered_redshift_df.loc[filtered_redshift_df['GRB'] == grb_name, 'alphaErr'] = alpha_err_value

    # Drop rows with missing values
    filtered_redshift_df = filtered_redshift_df.dropna()

    #Init an empty array
    filtered_redshift_df['beta'] = ['NaN']*len(filtered_redshift_df)

    # Add 'GRB' to the start of the 'GRB' column entries
    filtered_redshift_df['GRB'] = 'GRB' + filtered_redshift_df['GRB'].astype(str)

    # Split data into two DataFrames based on the presence of redshift
    with_redshift = filtered_redshift_df[filtered_redshift_df['Redshift_crosscheck']!='NaN']
    without_redshift = filtered_redshift_df[filtered_redshift_df['Redshift_crosscheck']=='NaN']

    # Save DataFrames to separate files
    with_redshift.to_csv(output_file_with_redshift, sep='\t', index=False)
    without_redshift.to_csv(output_file_without_redshift, sep='\t', index=False)

    print(str(len(with_redshift)) + ' GRBs with redshift.')
    print(str(len(without_redshift)) + ' GRBs without redshift.')
    return with_redshift, without_redshift



# Example usage
index_file_path = 'indexes.txt'  # Replace with the actual path
redshift_file_path = file_path  # Replace with the actual path
output_file_with_redshift = 'filtered_data_with_redshift.txt'
output_file_without_redshift = 'filtered_data_without_redshift.txt'

result_df = filter_redshift_by_grb_names(index_file_path, redshift_file_path, output_file_with_redshift, output_file_without_redshift)

24 GRBs with redshift.
5 GRBs without redshift.


/var/folders/8g/g48bxxwn3139k68mrwrpq2xw0000gn/T/ipykernel_20065/795218190.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_redshift_df['PhotonIndex'] = filtered_redshift_df['PhotonIndex'].apply(


In [32]:
## Finding set difference:

def read_grbs_from_file(file_path):
    with open(file_path, 'r') as file:
        return set(line.strip() for line in file)

def find_missing_grbs(large_file_path, small_file_path):
    # Read GRBs from both files
    large_grbs = read_grbs_from_file(large_file_path)
    small_grbs = read_grbs_from_file(small_file_path)
    print(len(small_grbs))
    # Find GRBs in the small list that are absent in the large list
    missing_grbs = small_grbs - large_grbs

    return missing_grbs

if __name__ == "__main__":
    # Replace 'large_file.txt' and 'small_file.txt' with your actual file paths
    large_file_path = 'redshifts.txt'
    small_file_path = 'indexes.txt'

    missing_grbs = find_missing_grbs(large_file_path, small_file_path)

    print("Missing GRBs: "+str(len(missing_grbs)))
    for grb in missing_grbs:
        print(grb)

70
Missing GRBs: 70
220521A
201221A
191019A
201021C
220117A
210323A
201014A
201216C
210517A
210731A
230328B
200219A
220219B
210619B
210411C
200613A
220107A
210726A
230506C
190829A
230204B
220627A
221009A
211023B
210919A
221110A
211207A
210722A
230818A
200826A
201103B
200522A
231117A
210222B
210610B
191004B
210504A
221226B
201020B
200411A
231118A
220611A
210210A
210321A
201024A
201020A
191031D
200524A
191221B
210822A
190919B
200829A
210610A
210104A
211024B
210420B
220527A
191011A
190719C
210204A
220101A
210702A
230414B
201221D
230325A
201015A
201104B
230116D
210905A
200205B
